In [26]:
import pandas as pd
from IPython.display import Image, HTML

In [27]:
# convert link to html tags 
def path_to_image_html(path):
    return '<img src="'+ path + '" width="60" >'

In [28]:
dfs = pd.read_csv("inputs/dfs.csv")
opt = pd.read_csv("inputs/optimal.csv")
st = pd.read_csv("inputs/statline.csv")
dfs = dfs.drop("Unnamed: 0", axis=1)
opt = opt.drop("Unnamed: 0", axis=1)
st = st.drop("Unnamed: 0", axis=1)

In [29]:
def gen_page(df, out):
    games = pd.read_csv('games.csv').loc[:,['name','slug']]
    trans_games = games.set_index('name').T
    img_dict = trans_games.to_dict('list')
    images = []
    for i in df["Name"]:
        code = "https://thumbs.dreamstime.com/b/creative-illustration-default-avatar-profile-placeholder-isolated-background-art-design-grey-photo-blank-template-mockup-144849704.jpg"
        if img_dict.get(i) != None:
            code = "https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/" + img_dict.get(i)[0] + ".jpg"
        images.append(code)
    df["Picture"] = images
    
    cols = df.columns.tolist()
    cols = [cols[0]] + cols[-1:] + cols[1:-1]
    df = df[cols]
    pd.set_option('display.max_colwidth', -1)
    df.index +=1
    pre = (
        "<head>\n<script src=\"https://kryogenix.org/code/browser/sorttable/sorttable.js\"></script>\n</head>\n\n<body>\n<div id=\"filter\">\n<input type=\"text\" id=\"name\" onkeyup=\"nameSearch()\" placeholder=\"Search for name...\">\n<input type=\"text\" id=\"team\" onkeyup=\"teamSearch()\" placeholder=\"Search for team...\">\n"
        "<input type=\"text\" id=\"position\" onkeyup=\"posSearch()\" placeholder=\"Search for position...\">\n"
        "</div>\n\n"
    )
    post = (
        "\n"
        "\n<script src=\"main.js\"></script>"
        "\n</body>"
        )
    # render dataframe as html
    html = HTML(df.to_html(escape=False ,formatters=dict(Picture=path_to_image_html)))
    body = html.data
    body = body.replace("dataframe", "sortable", 1)
    body = body.replace("class=\"sortable\"", "id=\"tbl\" class=\"sortable\"", 1)
    a = pre + body + post
    # write html to file
    with open(out, 'w', encoding="utf-8") as f:
        f.write(a)

In [30]:
gen_page(dfs, 'dfs.html')
gen_page(opt, 'opt.html')
gen_page(st, 'st.html')

C:\Users\kunal_tg\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  after removing the cwd from sys.path.
